# 要約 
このJupyterノートブックは、Kaggleの「LMSYS - Chatbot Arena」コンペティションで使用されるモデルを用いた人間の好み予測に取り組んでいます。具体的には、Gemma2とLlama3の大規模言語モデルを用いて、ユーザーからのプロンプトに対する両モデルの応答の勝率を予測するタスクに焦点を当てています。

### 主な問題
- **目的**: 2つの異なるチャットボット（GemmaとLlama）が生成した応答のどちらがユーザーに好まれるかを予測し、その勝率を計算すること。
- **評価**: モデルの出力冒険の確率を算出し、最終的に提出用データフレームを生成します。

### 使用されている手法とライブラリ
- **ライブラリ**: 
  - `transformers`: モデルとトークナイザーの操作に使用。
  - `peft`: LoRA（Low-Rank Adaptation）技術を使用してモデルの性能を向上させる。
  - `torch`: モデルの実行に必要なPyTorchライブラリ。
  - `sklearn`, `numpy`, `pandas`: データ処理と機械学習に関連するライブラリ。

- **手法**: 
  - GemmaとLlamaモデルをそれぞれ異なるGPUに配置し、並列処理を行っています。
  - テストデータはCSVファイルから読み込み、プロンプトと応答をトークン化し、モデルに供給するための処理を実施。
  - 推論では、両モデルが生成した応答に基づいて勝つ確率を算出し、結果を統合します。
  - 最後に、確率をCSVファイルとして出力し、Kaggleコンペティションに提出する準備を整えます。

このノートブックのアプローチは、ゲームの中での応答を比較し、どのモデルがユーザーの好みにより合致しているかを判定するための強力なテクニックを示しています。結果を提出するために最終的に生成されるCSVファイルは、ユーザーの選好をモデル化する精度に基づいて評価されます。

---


# 用語概説 
このノートブックに含まれる専門用語の中で、初心者がつまずきそうなものについて解説します。以下に挙げる用語は、特にマイナーまたは実務での経験がなければ馴染みが薄いものや、ノートブック特有のドメイン知識に関連するものです。

1. **PEFT (Parameter-Efficient Fine-Tuning)**:
   - モデルの全パラメータを更新するのではなく、一部のパラメータのみを調整することを目的とした技術。これにより、少ない計算リソースでモデルを微調整することが可能になる。

2. **LoRA (Low-Rank Adaptation)**:
   - 長大なモデルの微調整手法の一つで、モデルのパラメータ行列を低ランクの形式で表現し、それに基づいて微調整する。従来の方法よりもパラメータ数が少なく、計算量も削減できる。

3. **BitsAndBytes**:
   - モデルの重みを効率的に格納し、量子化を通じてメモリ使用量を削減するためのライブラリ。特に、8ビット量子化に関連した設定も含まれる。

4. **AutoTokenizer**:
   - Hugging FaceのTransformersライブラリに組み込まれた、モデルに最適なトークナイザーを自動的に選択・初期化するクラス。ユーザーが別のトークナイザーを選択する必要がないため、使い勝手が良い。

5. **Datasets**:
   - 機械学習や深層学習において、モデルのトレーニングや評価に用いるデータの集合。CSVファイルから直接読み込むなどの操作が行われる。

6. **アテンションマスク (Attention Mask)**:
   - トランスフォーマーモデルにおいて、入力中のどのトークンを注意したかを示すためのマスク。通常、パディングされたトークンには注意を向けないように設定される。

7. **torch.cuda.amp.autocast**:
   - 自動混合精度を有効にするデコレーター。精度を落としつつ計算を高速化し、GPUのメモリ使用量を削減するために用いられる。

8. **検証用データセット (Validation Dataset)**:
   - モデルの性能を評価するために使用されるデータセット。トレーニング中に使用されず、過学習を防ぐために重要。

9. **トークン化 (Tokenization)**:
   - 入力テキストをモデルが理解できる形式に変換するプロセス。テキストをトークン（単語やサブワード）に分割し、それぞれに数値的なIDを割り当てる。

10. **推論 (Inference)**:
    - 学習されたモデルを用いて、新しいデータに対して予測を行うプロセス。トレーニングと異なり、モデルの重みは更新されない。

これらの用語を理解することで、ノートブックや関連する機械学習・深層学習のトピックについての理解が深まることでしょう。

---


In [ ]:
"""
クレジット:

https://www.kaggle.com/code/emiz6413/llama-3-8b-38-faster-inference
https://www.kaggle.com/code/kishanvavdara/inference-llama-3-8b
https://www.kaggle.com/code/emiz6413/inference-gemma-2-9b-4-bit-qlora

リーダーボードスコア: 0.945
"""

In [ ]:
# 必要なライブラリをインストールします。
# transformers, peft, accelerate, bitsandbytesの最新バージョンを指定してインストールします。
# --no-indexオプションを使用することで、PyPI（Python Package Index）を使用せずに、
# 指定したリンクからのみパッケージを取得することを意味します。

!pip install transformers peft accelerate bitsandbytes -U --no-index --find-links /kaggle/input/lmsys-wheel-files

In [ ]:
# 必要なライブラリをインポートします。
import time  # タイマー処理用
from dataclasses import dataclass  # データクラスの作成用
from concurrent.futures import ThreadPoolExecutor  # スレッドプールによる非同期処理用

import torch  # PyTorchライブラリ
import sklearn  # 機械学習ライブラリ
import numpy as np  # 数値計算ライブラリ
import pandas as pd  # データ操作ライブラリ
from transformers import (
    Gemma2ForSequenceClassification,  # Gemma2モデルのシーケンス分類用クラス
    GemmaTokenizerFast,  # Gemmaモデル用のトークナイザー
    AutoTokenizer,  # 自動トークナイザー
    LlamaForSequenceClassification,  # Llamaモデルのシーケンス分類用クラス
    BitsAndBytesConfig  # BitsAndBytesの設定
)
from transformers.data.data_collator import pad_without_fast_tokenizer_warning  # トークナイザーの警告を抑制するためのデータコレータ
from peft import PeftModel, get_peft_model, LoraConfig, TaskType  # PEFTライブラリのモデル・設定用クラス

# メモリ効率の良いSDP（スパースディストリビューションプロセス）をCUDAバックエンドで有効にします。
torch.backends.cuda.enable_mem_efficient_sdp(True)
# フラッシュSDPも有効にします。
torch.backends.cuda.enable_flash_sdp(True)

In [ ]:
@dataclass
class Config:
    # Gemmaモデルのディレクトリパスを指定します。
    gemma_dir = '/kaggle/input/gemma-2/transformers/gemma-2-9b-it-4bit/1/gemma-2-9b-it-4bit'
    # GemmaのLoRAチェックポイントのディレクトリパスを指定します。
    gemma_lora_dir = '/kaggle/input/73zap2gx/checkpoint-5748'
    # Llamaモデルの名前（ディレクトリパス）を指定します。
    llama_model_name = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'
    # Llamaモデルの重みファイルのパスを指定します。
    llama_weights_path = '/kaggle/input/lmsys-model/model'
    # 最大シーケンス長を設定します（ここでは2048トークン）。
    max_length = 2048
    # バッチサイズを設定します（ここでは4）。
    batch_size = 4
    # テスト時のデータ拡張を使用するかどうかを指定します（ここでは使用しない）。
    tta = False
    # 最大シーケンス長の拡張を使用するかどうかを指定します（ここでは使用しない）。
    spread_max_length = False

# 新しいConfigオブジェクトを生成します。
cfg = Config()

In [ ]:
# テストデータセットをCSVファイルから読み込みます。
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

# テキストを処理する関数を定義します。
def process_text(text: str) -> str:
    # 評価関数を用いて、テキスト内の'null'を空の文字列に置き換えます。
    return " ".join(eval(text, {"null": ""}))

# 'prompt'列のテキストを処理します。
test.loc[:, 'prompt'] = test['prompt'].apply(process_text)
# 'response_a'列のテキストを処理します。
test.loc[:, 'response_a'] = test['response_a'].apply(process_text)
# 'response_b'列のテキストを処理します。
test.loc[:, 'response_b'] = test['response_b'].apply(process_text)

In [ ]:
# トークナイザーを使用して入力データをトークン化する関数を定義します。
def tokenize(tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length):
    # トークナイザーの種類によって処理を分けます。
    if isinstance(tokenizer, GemmaTokenizerFast):
        # Gemmaトークナイザーの場合は特定のプレフィックスを追加します。
        prompt = ["<prompt>: " + p for p in prompt]
        response_a = ["\n\n<response_a>: " + r_a for r_a in response_a]
        response_b = ["\n\n<response_b>: " + r_b for r_b in response_b]
    else:
        # その他のトークナイザーの場合は別のプレフィックスを追加します。
        prompt = ["User prompt: " + p for p in prompt]
        response_a = ["\n\nModel A :\n" + r_a for r_a in response_a]
        response_b = ["\n\n--------\n\nModel B:\n" + r_b for r_b in response_b]
    
    # max_lengthをスプレッドするかどうかで処理を分けます。
    if spread_max_length:
        # 各入力の長さを最大値の1/3に設定し、トークン化します。
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        # 入力IDsを結合します。
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        # アテンションマスクを作成します。
        attention_mask = [[1]* len(i) for i in input_ids]
    else:
        # 各入力をトークン化し、max_lengthに基づいて制限します。
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)
        input_ids = tokenized.input_ids
        attention_mask = tokenized.attention_mask
    # 入力IDsとアテンションマスクを返します。
    return input_ids, attention_mask

In [ ]:
# Gemmaトークナイザーを初期化します。
gemma_tokenizer = GemmaTokenizerFast.from_pretrained(cfg.gemma_dir)
# 終了トークンを追加することを指定します。
gemma_tokenizer.add_eos_token = True
# パディングを右側に設定します。
gemma_tokenizer.padding_side = "right"

# Llamaトークナイザーを初期化します。
llama_tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/lmsys-model/tokenizer')

# 両方のモデル用にデータを準備します。

# Gemma用のデータフレームを作成します。
gemma_data = pd.DataFrame()
# テストデータのIDを設定します。
gemma_data["id"] = test["id"]
# トークナイザーを使って入力IDsとアテンションマスクを取得します。
gemma_data["input_ids"], gemma_data["attention_mask"] = tokenize(gemma_tokenizer, test["prompt"], test["response_a"], test["response_b"])
# 各入力の長さを計算します。
gemma_data["length"] = gemma_data["input_ids"].apply(len)

# Llama用のデータフレームを作成します。
llama_data = pd.DataFrame()
# テストデータのIDを設定します。
llama_data["id"] = test["id"]
# トークナイザーを使って入力IDsとアテンションマスクを取得します。
llama_data["input_ids"], llama_data["attention_mask"] = tokenize(llama_tokenizer, test["prompt"], test["response_a"], test["response_b"])
# 各入力の長さを計算します。
llama_data["length"] = llama_data["input_ids"].apply(len)

In [ ]:
# GemmaモデルをGPU 0に読み込みます。
device_0 = torch.device('cuda:0')
gemma_model = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,  # Gemmaモデルのディレクトリを指定します。
    device_map=device_0,  # モデルを配置するデバイスを指定します。
    use_cache=False  # キャッシュを使用しないように設定します。
)
# LoRAモデルをGemmaモデルに読み込みます。
gemma_model = PeftModel.from_pretrained(gemma_model, cfg.gemma_lora_dir)  # LoRAチェックポイントのディレクトリを指定します。

In [ ]:
# LlamaモデルをGPU 1に読み込みます。
device_1 = torch.device('cuda:1')
# 8ビット量子化の設定を行います。
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,  # 8ビットでモデルをロードします。
    bnb_8bit_compute_dtype=torch.float16,  # 計算時のデータ型をfloat16に設定します。
    bnb_8bit_use_double_quant=False  # ダブル量子化を使用しない設定です。
)
# Llamaの基本モデルを読み込みます。
llama_base_model = LlamaForSequenceClassification.from_pretrained(
    cfg.llama_model_name,  # Llamaモデルの名前（ディレクトリ）を指定します。
    num_labels=3,  # 分類するラベルの数を設定します（ここでは3つのラベル）。
    torch_dtype=torch.float16,  # モデルのデータ型をfloat16に設定します。
    quantization_config=bnb_config,  # 量子化の設定を指定します。
    device_map='cuda:1'  # モデルを配置するデバイスを指定します。
)
# LlamaトークナイザーのパディングトークンIDを設定します。
llama_base_model.config.pad_token_id = llama_tokenizer.pad_token_id

# LoRAの設定を行います。
peft_config = LoraConfig(
    r=16,  # LoRAのランク。
    lora_alpha=32,  # LoRAのスケーリングファクター。
    lora_dropout=0.10,  # LoRAのドロップアウト率。
    bias='none',  # バイアスの設定（ここではなし）。
    inference_mode=True,  # 推論モードを有効にします。
    task_type=TaskType.SEQ_CLS,  # タスクの種類をシーケンス分類に設定します。
    target_modules=['o_proj', 'v_proj']  # ターゲットモジュールの名前を指定します。
)
# LoRAモデルを取得し、GPU 1に移動します。
llama_model = get_peft_model(llama_base_model, peft_config).to(device_1)
# Llamaモデルの重みを指定されたパスから読み込みます。
llama_model.load_state_dict(torch.load(cfg.llama_weights_path), strict=False)
# モデルを評価モードに設定します。
llama_model.eval()

In [ ]:
@torch.no_grad()  # 勾配の計算をオフにします（推論中は必要ないため）。
@torch.cuda.amp.autocast()  # 自動混合精度を有効にします（計算の効率を上げるため）。
def inference(df, model, tokenizer, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    # 各モデルの勝利確率を保存するリストを初期化します。
    a_win, b_win, tie = [], [], []
    
    # データフレームをバッチ単位で処理します。
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終わりのインデックスを計算します。
        tmp = df.iloc[start_idx:end_idx]  # 現在のバッチを取得します。
        input_ids = tmp["input_ids"].to_list()  # 入力IDsをリストとして取得します。
        attention_mask = tmp["attention_mask"].to_list()  # アテンションマスクをリストとして取得します。
        
        # トークナイザーを使って入力を整形します。
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},  # 入力IDsとアテンションマスクを指定します。
            padding="longest",  # 最長の入力にパディングします。
            pad_to_multiple_of=None,  # 特に指定しない場合はNoneにします。
            return_tensors="pt",  # PyTorchのテンソル形式で返します。
        )
        
        # モデルを使用して推論を行います。
        outputs = model(**inputs.to(device))  # 入力を指定されたデバイスに移動させてモデルに渡します。
        proba = outputs.logits.softmax(-1).cpu()  # ロジットからソフトマックスを適用して確率を計算します。
        
        # 各モデルの勝率をリストに追加します。
        a_win.extend(proba[:, 0].tolist())
        b_win.extend(proba[:, 1].tolist())
        tie.extend(proba[:, 2].tolist())
    
    # 勝率をデータフレームに追加します。
    df["winner_model_a"] = a_win
    df["winner_model_b"] = b_win
    df["winner_tie"] = tie
    
    return df  # 処理したデータフレームを返します。

In [ ]:
st = time.time()  # 処理開始時刻を記録します。

# 入力の長さでデータをソートします（長い順）。
gemma_data = gemma_data.sort_values("length", ascending=False)
llama_data = llama_data.sort_values("length", ascending=False)

# スレッドプールを使用して並列処理を実行します。
with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference, 
                           (gemma_data, llama_data),  # GemmaとLlamaのデータを引数として渡します。
                           (gemma_model, llama_model),  # 各モデルを引数として渡します。
                           (gemma_tokenizer, llama_tokenizer),  # 各トークナイザーを引数として渡します。
                           (device_0, device_1))  # 各モデルが配置されているデバイスを引数として渡します。

gemma_result_df, llama_result_df = list(results)  # 結果をリストに変換して変数に格納します。

# 結果を組み合わせます（簡単な平均を計算します）。
combined_result_df = gemma_result_df.copy()  # Gemmaの結果データフレームをコピーします。
combined_result_df["winner_model_a"] = (gemma_result_df["winner_model_a"] + llama_result_df["winner_model_a"]) / 2  # モデルAの勝率を平均します。
combined_result_df["winner_model_b"] = (gemma_result_df["winner_model_b"] + llama_result_df["winner_model_b"]) / 2  # モデルBの勝率を平均します。
combined_result_df["winner_tie"] = (gemma_result_df["winner_tie"] + llama_result_df["winner_tie"]) / 2  # 引き分け率を平均します。

# 推論にかかった時間を表示します。
print(f"Inference time: {time.time() - st:.2f} seconds")

In [ ]:
# 提出用のデータフレームを作成します。
# 'id'、'winner_model_a'、'winner_model_b'、'winner_tie'の列を選択します。
submission_df = combined_result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]
# データフレームをCSVファイルに書き出します（インデックスは含めません）。
submission_df.to_csv('submission.csv', index=False)
# 提出データの先頭5行を表示します。
display(submission_df.head())

---

# コメント

> ## Nikita Glazunov
> 
> こんにちは！良いノートブックです。Llama 3モデルを読み込もうとしたときに、このエラーが出ました >OSError: Incorrect path_or_model_id: '/kaggle/input/llama-3/transformers/8b-chat-hf/1'。ローカルフォルダへのパスまたはHub上のモデルのrepo_idを提供してください。どうすれば修正できますか？コードは一切変更していません。
> 
> 

---

> ## YEI0907
> 
> 推論にどれくらいの時間がかかりましたか？
> 
> 
> > ## G John Rao（トピック作成者）
> > 
> > 約9時間かかります。
> > 
> 
> 